In [ ]:
!pip install torch pandas numpy librosa huggingface_hub transformers evaluate soundfile accelerate

ERROR: Operation cancelled by user


In [ ]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; plat

In [ ]:
from datasets import load_dataset, Dataset
import torch, gc, os, json
import pandas as pd
import numpy as np
from transformers import (
    Wav2Vec2FeatureExtractor,
    AutoModelForAudioClassification,
    TrainerCallback,
    TrainingArguments,
    Trainer,
)
import evaluate
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve
from scipy.optimize import brentq
from scipy.interpolate import interp1d

In [ ]:
voxceleb1 = load_dataset("Codec-SUPERB/Voxceleb1_test_original")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Running on:", device)

README.md:   0%|          | 0.00/480 [00:00<?, ?B/s]

(…)-00000-of-00003-d0eea6966e0eba2f.parquet:   0%|          | 0.00/441M [00:00<?, ?B/s]

(…)-00001-of-00003-1ab27c4d03da1754.parquet:   0%|          | 0.00/412M [00:00<?, ?B/s]

(…)-00002-of-00003-4d6815b5778a4368.parquet:   0%|          | 0.00/435M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4874 [00:00<?, ? examples/s]

Running on: cuda:0


In [ ]:
waveform, sr, id_list = [], [], []
for dp in voxceleb1["test"]:
    waveform.append(dp["audio"]["array"])
    sr.append(dp["audio"]["sampling_rate"])
    id_list.append(dp["id"])

In [ ]:
def preprocess_id(id_):
    idx = id_.find("+")
    return id_[:idx]

cleaned_id = [preprocess_id(i) for i in id_list]
df = pd.DataFrame({"audio": waveform, "id": cleaned_id})
hf_dataset_from_df = Dataset.from_pandas(df)

In [ ]:
target_length = 16000

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base")

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(example):
    audio_array = example["audio"]
    cur_len = len(audio_array)

    if cur_len < target_length:
        audio_array = np.pad(audio_array, (0, target_length - cur_len), "constant")
    elif cur_len > target_length:
        audio_array = audio_array[:target_length]

    inputs = feature_extractor(
        audio_array, sampling_rate=16000, return_tensors="pt", padding="longest"
    )
    return {"input_values": inputs.input_values.squeeze(0)}

In [ ]:
label_list = sorted(set(cleaned_id))
num_labels = len(label_list)
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for l, i in label2id.items()}

def convert_labels_to_ids(example):
    example["label_ids"] = label2id[example["id"]]
    return example

In [ ]:
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    labels = eval_pred.label_ids

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    precision = precision_score(labels, predictions, average="macro", zero_division=0)
    recall = recall_score(labels, predictions, average="macro", zero_division=0)
    f1 = f1_score(labels, predictions, average="macro", zero_division=0)

    # Multi-class EER (one-vs-rest)
    eer_vals = []
    for cls in np.unique(labels):
        bin_labels = (labels == cls).astype(int)
        probs = eval_pred.predictions[:, cls]
        fpr, tpr, _ = roc_curve(bin_labels, probs)
        if len(fpr) > 1:
            eer = brentq(lambda x: 1.0 - x - interp1d(fpr, tpr)(x), 0.0, 1.0)
            eer_vals.append(eer)
        else:
            eer_vals.append(0.0)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "eer": np.mean(eer_vals),
    }

In [ ]:
def nll_loss(logits, labels):
    return F.cross_entropy(logits, labels.long())

class CustomTrainer(Trainer):  # keep extra arg for safety
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        loss = nll_loss(outputs.logits, labels)
        return (loss, outputs) if return_outputs else loss

def minimal_data_collator(features):
    inputs = torch.stack([torch.as_tensor(f["input_values"]) for f in features]).to(device)
    labels = torch.tensor([f["label_ids"] for f in features]).to(device)
    return {"input_values": inputs, "labels": labels}

In [ ]:
target_lengths = [
    16000, 32000, 48000, 63360, 64000, 80000, 96000, 103680,
    112000, 128000, 132400, 160000, 240000, 480000, 800000, 1104640
]

In [ ]:
class JsonLoggerCallback(TrainerCallback):
    def __init__(self, filepath):
        self.filepath = filepath
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics is None:   # just in case HF calls with None
            return
        record = {"epoch": state.epoch}
        record.update(metrics)
        with open(self.filepath, "a") as f:
            f.write(json.dumps(record) + "\n")

In [ ]:
for tl in target_lengths:
    print(f"\n Starting run for target_length = {tl}")
    target_length = tl  # <-- updates the global used by preprocess_function

    # Re-encode / label-encode the dataset
    proc_ds = hf_dataset_from_df.map(
        preprocess_function, remove_columns=["audio"], desc=f"Padding to {tl}"
    ).map(convert_labels_to_ids, remove_columns=["id"])
    split_ds = proc_ds.train_test_split(test_size=0.2, seed=42)
    split_ds.set_format("torch")   # leave on CPU; collator moves to GPU

    # Fresh model each time
    model = AutoModelForAudioClassification.from_pretrained(
        "facebook/wav2vec2-base",
        num_labels=num_labels,
        label2id=label2id,
        id2label=id2label,
        ignore_mismatched_sizes=True,
    ).to(device)

    # HF args -- no checkpoint saving
    args = TrainingArguments(
        output_dir=f"./tmp_{tl}",           # temp dir
        eval_strategy="epoch",
        save_strategy="no",                # ← nothing saved
        learning_rate=3e-5,
        per_device_train_batch_size=32,
        gradient_accumulation_steps=4,
        per_device_eval_batch_size=32,
        num_train_epochs=20,
        warmup_ratio=0.1,
        logging_strategy="epoch",
        metric_for_best_model="eer",
        greater_is_better=False,
        dataloader_pin_memory=False,
        remove_unused_columns=False,
        report_to="none",
        push_to_hub=False,
    )

    json_path = f"eval_{tl}.jsonl"
    if os.path.exists(json_path):     # start fresh
        os.remove(json_path)

    trainer = CustomTrainer(
        model=model,
        args=args,
        train_dataset=split_ds["train"],
        eval_dataset=split_ds["test"],
        compute_metrics=compute_metrics,
        data_collator=minimal_data_collator,
        callbacks=[JsonLoggerCallback(json_path)],
    )

    trainer.train()

    # tidy GPU / RAM before next run
    del model, trainer, split_ds, proc_ds
    torch.cuda.empty_cache()
    gc.collect()

print("\n All lengths completed!  Check the `eval_<length>.jsonl` files.")


 Starting run for target_length = 16000


Padding to 16000:   0%|          | 0/4874 [00:00<?, ? examples/s]

Map:   0%|          | 0/4874 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


...


 Starting run for target_length = 480000


Padding to 480000:   0%|          | 0/4874 [00:00<?, ? examples/s]

Map:   0%|          | 0/4874 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.86 GiB. GPU 0 has a total capacity of 39.56 GiB of which 166.88 MiB is free. Process 5321 has 39.38 GiB memory in use. Of the allocated memory 24.45 GiB is allocated by PyTorch, and 14.44 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
target_lengths_continued = [480000,  800000, 1104640]

for tl in target_lengths_continued:
    print(f"\n Starting run for target_length = {tl}")
    target_length = tl  # <-- updates the global used by preprocess_function

    # Re-encode / label-encode the dataset
    proc_ds = hf_dataset_from_df.map(
        preprocess_function, remove_columns=["audio"], desc=f"Padding to {tl}"
    ).map(convert_labels_to_ids, remove_columns=["id"])
    split_ds = proc_ds.train_test_split(test_size=0.2, seed=42)
    split_ds.set_format("torch")   # leave on CPU; collator moves to GPU

    # Fresh model each time
    model = AutoModelForAudioClassification.from_pretrained(
        "facebook/wav2vec2-base",
        num_labels=num_labels,
        label2id=label2id,
        id2label=id2label,
        ignore_mismatched_sizes=True,
    ).to(device)
    model.gradient_checkpointing_enable()

    # HF args -- no checkpoint saving
    args = TrainingArguments(
        output_dir=f"./tmp_{tl}",           # temp dir
        eval_strategy="epoch",
        save_strategy="no",                # ← nothing saved
        learning_rate=3e-5,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=64,
        per_device_eval_batch_size=1,
        num_train_epochs=20,
        warmup_ratio=0.1,
        bf16=True,
        gradient_checkpointing=True,
        logging_strategy="epoch",
        metric_for_best_model="eer",
        greater_is_better=False,
        dataloader_pin_memory=False,
        remove_unused_columns=False,
        report_to="none",
        push_to_hub=False,
    )

    json_path = f"eval_{tl}.jsonl"
    if os.path.exists(json_path):     # start fresh
        os.remove(json_path)

    trainer = CustomTrainer(
        model=model,
        args=args,
        train_dataset=split_ds["train"],
        eval_dataset=split_ds["test"],
        compute_metrics=compute_metrics,
        data_collator=minimal_data_collator,
        callbacks=[JsonLoggerCallback(json_path)],
    )

    trainer.train()

    # tidy GPU / RAM before next run
    del model, trainer, split_ds, proc_ds
    torch.cuda.empty_cache()
    gc.collect()

print("\n All lengths completed!  Check the `eval_<length>.jsonl` files.")


 Starting run for target_length = 480000


Padding to 480000:   0%|          | 0/4874 [00:00<?, ? examples/s]

Map:   0%|          | 0/4874 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


...

Padding to 1104640:   0%|          | 0/4874 [00:00<?, ? examples/s]

Map:   0%|          | 0/4874 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Eer
1,3.592000,3.603732,0.093333,0.021569,0.040687,0.013370,0.351102
2,3.434100,3.324852,0.181538,0.075714,0.093680,0.044248,0.194237
3,3.154800,3.024584,0.355897,0.210280,0.228378,0.177616,0.113984
4,2.863400,2.739486,0.529231,0.365187,0.380769,0.327622,0.086848
5,2.562900,2.439880,0.603077,0.400984,0.453104,0.385423,0.064482
6,2.276300,2.156895,0.668718,0.446938,0.499582,0.438178,0.055362
7,2.014500,1.931453,0.731282,0.548256,0.578841,0.528504,0.045980
8,1.794200,1.720076,0.755897,0.592053,0.603912,0.560578,0.025067
9,1.606900,1.548928,0.832821,0.740777,0.713620,0.686813,0.026389
10,1.438500,1.370541,0.887179,0.861451,0.801565,0.797836,0.015760



 All lengths completed!  Check the `eval_<length>.jsonl` files.
